In [1]:
# Advent of code
# Day 18

with open('input.txt') as f:
    data = f.read()

data = data.split('\n')
data = list(filter(None, data))
for i, d in enumerate(data):
    data[i] = d.split(',')
    data[i] = [int(x) for x in data[i]]
    

In [2]:
# get unconnected sides for a single cube
def get_unconnected_sides(i):
    # neighbors in the x direction:  y and Z are the same, X is off by 1
    x_neighbors = [n for n in data if abs(i[0]-n[0]) == 1 and i[1]==n[1] and i[2]==n[2]]
    y_neighbors = [n for n in data if abs(i[1]-n[1]) == 1 and i[0]==n[0] and i[2]==n[2]]
    z_neighbors = [n for n in data if abs(i[2]-n[2]) == 1 and i[0]==n[0] and i[1]==n[1]]
    all_neighbors = x_neighbors + y_neighbors + z_neighbors

    unconnected_sides = 6-len(all_neighbors)
    return unconnected_sides


In [3]:
# total unconnected sides for all cubes
total_unconnected_sides = 0

for i in data:
    total_unconnected_sides += get_unconnected_sides(i)

print(total_unconnected_sides)




4314


In [4]:
data[:5]

[[12, 13, 19], [13, 5, 4], [11, 8, 1], [8, 6, 3], [19, 9, 12]]

In [5]:
# Part 2: Determine the outside surface area only,
# not including any interior air pockets in the lava

# Set up boundaries for a 3d area around
# the lava, with a bufer on all sides
min_x = min(p[0] for p in data)-2
max_x = max(p[0] for p in data)+2

min_y = min(p[1] for p in data)-2
max_y = max(p[1] for p in data)+2

min_z = min(p[2] for p in data)-2
max_z = max(p[2] for p in data)+2


In [6]:
# build a matrix containing the entire lava blob
matrix = {}

for x in range(min_x, max_x+1):
    for y in range(min_y, max_y+1):
        for z in range(min_z, max_z+1):
            matrix[(x,y,z)]=''



In [7]:

# identify the lava points in the matrix (not
# interior air pockets or areas outside the lava)
for x,y,z in data:
    matrix[(x,y,z)]='lava'


In [8]:
# Flood the area outside the lava block and identify
# those points. All points left unidentified  after this step 
# are voids inside the lava.

def flood_matrix(x,y,z):
    to_check = [(x,y,z)]
    checked = []
    def flood(x,y,z):
        while len(to_check) > 0:
            (x,y,z) = to_check.pop()
            if matrix[(x,y,z)] == 'lava':
                checked.append((x,y,z))
            elif matrix[(x,y,z)] != '':
                checked.append((x,y,z))
            else:
                matrix[(x,y,z)] = 'outside'
                neighbors = [
                    (x-1, y, z),(x+1, y, z),
                    (x, y-1, z), (x, y+1, z),
                    (x, y, z-1), (x, y, z+1)]

                for n in neighbors:
                    if (n[0],n[1],n[2]) in list(matrix.keys()) and ((n[0],n[1],n[2]) not in checked):
                        # print('neighbor found')
                        to_check.append(n)
                checked.append((x,y,z))
    flood(x,y,z)
    print(f'{len(checked)} checked locations')
    print(f'{len(to_check)} unchecked locations')

# fill starting with a point outside the solid area
flood_matrix(-1,-2,-2)


34538 checked locations
0 unchecked locations


In [17]:
# for interior void points, find the sides NOT connected to solid cubes, if any.
# Those would be connected to other interior void points, so they did not appear
# as surface squares in Part 1. They will not need to be subtracted
# from the total when removing interior surfaces. 

# interior void points are those not identified as lava or outside
interior_points = [p for p in matrix if matrix[p]=='']
print(len(interior_points))
print(len(matrix))

unconnected_sides_interior = 0
for p in interior_points:
    unconnected_sides_interior += get_unconnected_sides(p)

total_interior_surface = (len(interior_points) * 6) - unconnected_sides_interior

total_outside_surface = total_unconnected_sides - total_interior_surface

print(total_outside_surface)

1447
16224
2444
